# Importando as Bibliotecas 

In [84]:
import numpy as np
from glob import glob
import os
from skimage.feature import blob_dog, blob_log, blob_doh
import pydicom
from skimage.feature import hessian_matrix, hessian_matrix_det
from sklearn.preprocessing import StandardScaler
from skimage.io import imread, imread_collection
from skimage.color import rgb2gray
import matplotlib.pyplot as plt 
from skimage.transform import rotate
from skimage import data
from scipy.stats import skew
from scipy.stats import kurtosis
import scipy as sp
import time 
import scipy.stats as sps
import pandas as pd 

%matplotlib inline

# 1ª Etapa 

## Lendo as imagens da base (Benignos)

In [85]:
data_path = "base/"
output_path = "base/csv/"

In [86]:
def lendo_imagens(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)] 
    return(slices)

In [87]:
# Conjunto de imagens 
data_benigno = lendo_imagens(data_path+'benigno')
data_maligno = lendo_imagens(data_path+'maligno')

## Teste com imagens .png

In [88]:
# Conjunto de imagens 
data_dog = imread_collection(data_path+'dog/*.jpg')
data_cat = imread_collection(data_path+'cat/*.jpg')

# det(Happrox)= DxxDyy−(0.9Dxy)²

## 1ª Etapa 

In [89]:
def etapa_01_log(image):  
    #Laplacian of Gaussian #Difference of Gaussian #Determinant of Hessian
    return(blob_log(image, max_sigma=0.9, threshold=.001))

In [90]:
def etapa_01_dog(image):  
    #Difference of Gaussian
    return(blob_dog(image, max_sigma=0.9, threshold=.001))

In [91]:
def etapa_01_doh(image):  
    #Determinant of Hessian
    return(blob_doh(image,  max_sigma=0.2, threshold=.0001))

# 2ª Etapa 

## Definindo bordas na imagem

In [92]:
def borda(img_orig, filtro):
    x = int(img_orig.shape[0])
    y = int(img_orig.shape[1])
    # Calcula o valor de A e B referente ao filtro 
    a = int((filtro - 1) / 2)
    b = int((filtro - 1) / 2)
    # Gera uma matriz com bordas, todos elementos zerados 
    matriz_borda = np.zeros([x + a*2, y + b*2], dtype=img_orig.dtype)
    matriz_borda[a:x+a, b:y+b] = img_orig.copy()
    return(matriz_borda)

## Gerando a janela para calcular o Histograma 

In [93]:
def gerar_histo_janela(matriz_bordas, filtro, x, y):
    a = int((filtro - 1) / 2)
    b = int((filtro - 1) / 2)
    x1 = int(x) + a
    y1 = int(y) + b  
    #print('a: ', a, ' b: ', b)
    matriz_hist = matriz_bordas[(x1 - a): (x1 + a) + a, (y1 - b): (y1 + b) + b]
    return(matriz_hist)

## Calculando o Histograma para cada ponto da imagem

In [94]:
#Função que retorna os atributos de uma imagem 
def calcula_atributos(matriz, tam_janela, lista_pontos):  
    atributos = []
    atributos_mean = []
    atributos_var = []
    atributos_skew= []
    atributos_kurtosis = []
    atributos_power= []
    #atributos_entropy = [] Atributo classificado como ruim
    if len(lista_pontos) == 0:
        histograma = np.histogram(matriz.ravel(), bins=256)[0]
        atributos_mean.append(histograma.mean())
        atributos_var.append( histograma.var())
        atributos_skew.append(skew(histograma))
        atributos_kurtosis.append(kurtosis(histograma))
        atributos_power.append(sp.power(histograma,2).sum())
    for pontos in lista_pontos:
        y, x, _ = pontos
        janela = gerar_histo_janela(matriz, tam_janela, x, y)
        histograma = np.histogram(janela.ravel(), bins=256)[0]
        atributos_mean.append(histograma.mean())
        atributos_var.append( histograma.var())
        atributos_skew.append(skew(histograma))
        atributos_kurtosis.append(kurtosis(histograma))
        atributos_power.append(sp.power(histograma,2).sum())
        #atributos_entropy.append(sps.entropy(histograma))  Atributo classificado como ruim 
    atributos.append(np.mean(atributos_mean))
    atributos.append(np.mean(atributos_var))
    atributos.append(np.mean(atributos_skew))
    atributos.append(np.mean(atributos_kurtosis))
    atributos.append(np.mean(atributos_power))
    #atributos.append(np.mean(atributos_entropy)) Atributo classificado como ruim 
    return(atributos)

## Salvar CSV 

In [95]:
def csv_bm(matriz_b, matriz_m, name_file):  
    nomes_tributos = ['Media', 'Variancia', 'Skewness', 'Kurtosis','Energia', 'Classe']
    matriz_b_csv = pd.DataFrame(matriz_b, columns = nomes_tributos)
    matriz_m_csv = pd.DataFrame(matriz_m, columns = nomes_tributos)
    matriz_csv = pd.concat([matriz_b_csv, matriz_m_csv])
    matriz_csv.to_csv(output_path+name_file+'.csv', index=False)
    #print('CSV : [ '+ name_file+ ' ] gerado com sucesso!')

## Função para executar a 2ª ETAPA (todas as imagens da base .DCM)  

In [96]:
def etapa_02_dcm(base, calsse, tam_janela, num_funcao):
    qtd_pontos = 3 # Quantidade de pontos 
    linha = len(base) # Quantidade de linhas (imagens) 
    coluna = 6 # Quantidade de Atributos 
    matriz_result = np.zeros((linha, coluna)) # Definindo a matriz resultante (CSV)
    lista_pontos = []
    #print('shape: ', matriz_result.shape)
    for i in range(0, linha):
        if len(base[i].pixel_array.shape) < 3: 
            imagem = base[i].pixel_array
        else: 
            imagem = base[i].pixel_array[0]
        
        if num_funcao == 1: # ( 1 ) Laplacian of Gaussian
            lista_pontos = etapa_01_log(imagem) 
        elif num_funcao == 2: # ( 2 ) Difference of Gaussian
            lista_pontos = etapa_01_dog(imagem) 
        elif num_funcao == 3: # ( 3 ) Determinant of Hessian
             lista_pontos = etapa_01_dog(imagem) 
        
        matriz_bordas = borda(imagem, tam_janela) # Gerar a matriz com bordas, Janela 3x3

        atributos = calcula_atributos(matriz_bordas, tam_janela, lista_pontos) # Função que retorna os atributos de uma imagem

        matriz_result[i][0:-1] = atributos
        
        matriz_result[i][-1] = calsse
    return(matriz_result)

## Executar a função Etapa 02 (base de imagem, classe, janela, tipo de função) .DCM

### ( 1 ) Laplacian of Gaussian 
### ( 2 ) Difference of Gaussian 
### ( 3 ) Determinant of Hessian

## Função para executar etapa02 e gerar CSV .DCM

In [97]:
def main(benigno, maligno, janela):
    # ( 1 ) Laplacian of Gaussian 
    mtb = etapa_02_dcm(benigno, 0, janela, 1)  
    mtm = etapa_02_dcm(maligno, 1, janela, 1) 
    csv_bm(mtb, mtm, 'database_log_'+str(janela))

    # ( 2 ) Difference of Gaussian 
    mtb = etapa_02_dcm(benigno, 0, janela, 2)  
    mtm = etapa_02_dcm(maligno, 1, janela, 2) 
    csv_bm(mtb, mtm, 'database_dog_'+str(janela))

    # ( 3 ) Determinant of Hessian
    mtb = etapa_02_dcm(benigno, 0, janela, 3)  
    mtm = etapa_02_dcm(maligno, 1, janela, 3) 
    csv_bm(mtb, mtm, 'database_doh_'+str(janela))


## Função Principal (det(Happrox)= DxxDyy−(0.9Dxy)²)  .DCM

In [98]:
inicio = time.time()
# Janela 3x3 
main(data_benigno, data_maligno, 3)

# Janela 5x5
main(data_benigno, data_maligno, 5)

# Janela 7x7 
main(data_benigno, data_maligno, 7)

# Janela 9x9 
main(data_benigno, data_maligno, 9)
fim = time.time()
print('Tempo de Execução: ', fim-inicio)

Tempo de Execução:  128.2976038455963


# Função para executar a 2ª ETAPA (todas as imagens da base .PNG, JPG, Outras)

In [79]:
def etapa_02_all(base, calsse, tam_janela, num_funcao, canal):
    qtd_pontos = 3 # Quantidade de pontos 
    linha = len(base) # Quantidade de linhas (imagens) 
    coluna = 6 # Quantidade de Atributos 
    matriz_result = np.zeros((linha, coluna)) # Definindo a matriz resultante (CSV)
    lista_pontos = []
    #print('shape: ', matriz_result.shape)
    for i, img in enumerate(base):    
        img_ciza = rgb2gray(img[:,:, canal])
        if num_funcao == 1: # ( 1 ) Laplacian of Gaussian
            lista_pontos = etapa_01_log(img_ciza) 
        elif num_funcao == 2: # ( 2 ) Difference of Gaussian
            lista_pontos = etapa_01_dog(img_ciza) 
        elif num_funcao == 3: # ( 3 ) Determinant of Hessian
             lista_pontos = etapa_01_dog(img_ciza) 
        
        matriz_bordas = borda(img_ciza, tam_janela) # Gerar a matriz com bordas, Janela 3x3

        atributos = calcula_atributos(matriz_bordas, tam_janela, lista_pontos) # Função que retorna os atributos de uma imagem

        matriz_result[i][0:-1] = atributos
        
        matriz_result[i][-1] = calsse
    return(matriz_result)

## Executar a função Etapa 02 (base de imagem, classe, janela, tipo de função) .PNG, .JPG

### ( 1 ) Laplacian of Gaussian 
### ( 2 ) Difference of Gaussian 
### ( 3 ) Determinant of Hessian

In [78]:
def main_all(benigno, maligno, janela, canal):
    # ( 1 ) Laplacian of Gaussian 
    mtb = etapa_02_all(benigno, 0, janela, 1, canal)  
    mtm = etapa_02_all(maligno, 1, janela, 1, canal) 
    csv_bm(mtb, mtm, 'database_log_all_'+str(janela))

    # ( 2 ) Difference of Gaussian 
    mtb = etapa_02_all(benigno, 0, janela, 2, canal)  
    mtm = etapa_02_all(maligno, 1, janela, 2, canal) 
    csv_bm(mtb, mtm, 'database_dog_all_'+str(janela))

    # ( 3 ) Determinant of Hessian
    mtb = etapa_02_all(benigno, 0, janela, 3, canal)  
    mtm = etapa_02_all(maligno, 1, janela, 3, canal) 
    csv_bm(mtb, mtm, 'database_doh_all_'+str(janela))

## Função Principal (det(Happrox)= DxxDyy−(0.9Dxy)²)  .PNG, .JPG

In [99]:
# Janela 3x3, Canal 0 -> Gera 3 arquivos CSV 
main_all(data_cat, data_dog, 3, 0) # parâmetos (base_benigno, base_maligno, janela, canal)

# Janela 3x3, Canal 1 -> Gera 3 arquivos CSV 
main_all(data_cat, data_dog, 3, 1) # parâmetos (base_benigno, base_maligno, janela, canal)

# Janela 3x3, Canal 2 -> Gera 3 arquivos CSV 
main_all(data_cat, data_dog, 3, 2) # parâmetos (base_benigno, base_maligno, janela, canal)